In [1]:
# Reads in photometry from different sources, normalizes them, and puts them
# onto a BJD time scale

# Created 2021 Dec. 28 by E.S.

In [1]:
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib qt

In [2]:
## USER INPUTS

photometry_source_file = "bh_peg_aavso_prepolished.txt"
epoch_writeout_file = "junk_jds_bh_peg_aavso.csv" # generated as-we-go
bjd_file = "bh_peg_aavso_bjd.csv" # to be read in after manual correction
writeout_polished_file = "bh_peg_aavso_polished.txt"

In [3]:
stem_super = "/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/"
stem_moore_macadam = stem_super + "polished_macadam_moore_photometry/"
stem_public_sources = stem_super + "all_photometry_program_stars/polished/"
bjd_sources = stem_super + "all_photometry_program_stars/bjds/"

In [4]:
# read in photometry

df = pd.read_csv(stem_public_sources + photometry_source_file)
#df = pd.read_csv(stem_public_sources + photometry_source_file, delim_whitespace=True)

In [5]:
# if there's a bunch of too-old stuff, remove it
#df = df.where(np.logical_and(df["JD"]>2456474-400,df["JD"]<2456474+400)).dropna(how="all").reset_index()
#df = df.where(df["JD"]<=2459238.567486600).dropna(how="all").reset_index()

In [6]:
df.keys()

Index(['JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit'],
      dtype='object')

In [7]:
# if HJD minus the 245 in front
#df["HJD"] = np.add(2450000,df["HJD"])

In [8]:
# write out only JD, for conversion

#df["JD"] = np.add(df["obsmjd"],2400000.5)
df["JD"].to_csv(epoch_writeout_file, header=False, index=False)
#df["HJD"].to_csv(epoch_writeout_file, header=False, index=False)
print(epoch_writeout_file)

junk_jds_bh_peg_aavso.csv


# at this point, do conversion step manually online

In [9]:
# FYI: location of Wichita (approx. center of U.S.)

'''
lat_wichita = 37.688889
lon_wichita = -97.336111
'''

'\nlat_wichita = 37.688889\nlon_wichita = -97.336111\n'

In [11]:
len(intermed_bjd)

29428

# read bjds back in and tack onto the dataframe

In [9]:
intermed_bjd = pd.read_csv(bjd_sources + bjd_file, names=["BJD"])
print(bjd_sources + bjd_file)

df["BJD"] = intermed_bjd.values

#df = df.join(intermed_bjd["BJD"])

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/bjds/bh_peg_aavso_bjd.csv


In [10]:
len(df)

7384

In [11]:
df

,JD,Magnitude,Uncertainty,HQuncertainty,Band,Observer Code,Comment Code(s),Comp Star 1,Comp Star 2,Charts,...,Kmag,HJD,Star Name,Observer Affiliation,Measurement Method,Grouping Method,ADS Reference,Digitizer,Credit,BJD
0,2.453539e+06,9.696,0.008,NaN,V,DKS,G,106,NaN,GSC,...,NaN,NaN,BH PEG,NaN,STD,NaN,NaN,NaN,NaN,2.453539e+06
1,2.453539e+06,9.688,0.008,NaN,V,DKS,G,106,NaN,GSC,...,NaN,NaN,BH PEG,NaN,STD,NaN,NaN,NaN,NaN,2.453539e+06
2,2.453539e+06,9.673,0.008,NaN,V,DKS,G,106,NaN,GSC,...,NaN,NaN,BH PEG,NaN,STD,NaN,NaN,NaN,NaN,2.453539e+06
3,2.453539e+06,9.662,0.008,NaN,V,DKS,G,106,NaN,GSC,...,NaN,NaN,BH PEG,NaN,STD,NaN,NaN,NaN,NaN,2.453539e+06
4,2.453539e+06,9.654,0.008,NaN,V,DKS,G,106,NaN,GSC,...,NaN,NaN,BH PEG,NaN,STD,NaN,NaN,NaN,NaN,2.453539e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7379,2.459463e+06,10.271,0.005,NaN,V,SAH,NaN,ENSEMBLE,NaN,X23184FNT,...,NaN,NaN,BH PEG,AAVSO,STD,NaN,NaN,NaN,NaN,2.459463e+06
7380,2.459463e+06,10.273,0.004,NaN,V,SAH,NaN,ENSEMBLE,NaN,X23184FNT,...,NaN,NaN,BH PEG,AAVSO,STD,NaN,NaN,NaN,NaN,2.459463e+06
7381,2.459463e+06,10.257,0.006,NaN,V,SAH,NaN,ENSEMBLE,NaN,X23184FNT,...,NaN,NaN,BH PEG,AAVSO,STD,NaN,NaN,NaN,NaN,2.459463e+06
7382,2.459463e+06,10.281,0.004,NaN,V,SAH,NaN,ENSEMBLE,NaN,X23184FNT,...,NaN,NaN,BH PEG,AAVSO,STD,NaN,NaN,NaN,NaN,2.459463e+06


In [14]:
# in case of weird naming conventions

df["Magnitude"] = df["MAG_0"]
df["Uncertainty"] = df["MER_0"]


# look at the data, choose what we want

In [12]:
plt.clf()
plt.scatter(df["BJD"],
            df["Magnitude"])
plt.show()

In [11]:
plt.clf()
plt.scatter(df["BJD"],
            df["MAG_0"])
plt.scatter(df["BJD"],
            df["MAG_1"])
plt.scatter(df["BJD"],
            df["MAG_2"])
plt.scatter(df["BJD"],
            df["MAG_3"])
plt.scatter(df["BJD"],
            df["MAG_4"])
plt.show()

In [14]:
plt.clf()
plt.scatter(df["BJD"],
            df["mag_autocorr"])
plt.show()

In [15]:
plt.clf()
plt.scatter(df["JD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="k")
plt.scatter(df["BJD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="b")
plt.show()

In [15]:
plt.clf()
plt.scatter(df["JD"].where(df["Band"] == "V"),
            df["Magnitude"].where(np.logical_and(df["Observer Code"] == "SAH",df["Band"] == "V")), color="k")
plt.scatter(df["BJD"].where(df["Band"] == "V"),
            df["Magnitude"].where(np.logical_and(df["Observer Code"] == "SAH",df["Band"] == "V")), color="b")
plt.show()

In [18]:
plt.clf()
plt.scatter(df["BJD"].where(df["Observer Code"] == "SAH"),
            df["Magnitude"].where(df["Observer Code"] == "SAH"), color="k")
plt.show()

In [13]:
df.keys()

Index(['JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit', 'BJD'],
      dtype='object')

In [12]:
print(np.min(df["JD"]))
print(np.max(df["JD"]))

2439144.571
2459546.39706


In [11]:
len(df)

30529

In [13]:
df["Band"].value_counts()

V     6879
TG     504
TB       1
Name: Band, dtype: int64

In [14]:
df["Observer Code"].value_counts()

SAH     2464
DKS     1993
SRIC    1981
PLN      393
MZK      290
PTOB     134
CGRD      88
KCSB      22
TRE       16
JPG        2
SMRC       1
Name: Observer Code, dtype: int64

In [16]:
# choose photometric subset

#df_subset_photo = df
#df_subset_photo = df.where(np.logical_and(df["BJD"] > 2452456,df["Band"] == "V"))
#df_subset_photo = df.where(np.logical_or(df["BJD"] < 2455109,df["BJD"] > 2455535))
#df_subset_photo = df.where(np.logical_or(df["BJD"] < 2457764,df["BJD"] > 2457887))
df_subset_photo = df.where(np.logical_and(df["Band"] == "V",df["Observer Code"] == "SAH"))
#df_subset_photo = df.where(df["Band"] == "V")
#df_subset_photo = df.where(df["MAG_0"] < 20)
#df_subset = df[["JD", "sdf"]]

In [17]:
# additional winnowing

df_subset_photo = df_subset_photo.where(df_subset_photo["BJD"] > 2456024)

In [18]:
# check to confirm
plt.clf()
plt.scatter(df_subset_photo["BJD"],df_subset_photo["Magnitude"])
#plt.scatter(df_subset_photo["BJD"],df_subset_photo["MAG_0"])
plt.show()

In [18]:
len(df_subset_photo)

30529

In [18]:
np.max(df_subset_photo["BJD"])

2455167.640255

In [19]:
df_subset_photo.keys()

Index(['HJD', 'MAG_3', 'MAG_0', 'MAG_1', 'MAG_2', 'MAG_4', 'MER_3', 'MER_0',
       'MER_1', 'MER_2', 'MER_4', 'GRADE', 'FRAME', 'BJD', 'Magnitude',
       'Uncertainty'],
      dtype='object')

# feature scale the photometry

### wait! scaling of error bars won't be the same, since an offset is being subtracted off

In [29]:
'''
scaler = MinMaxScaler() # min-max mapped to 0-to-1
df_subset_photo[['mag_scaled']] = scaler.fit_transform(df_subset_photo[['Magnitude']])
# df_subset_photo[['error_scaled']] = scaler.transform(df_subset_photo[['Uncertainty']])
'''

In [21]:
'''
plt.scatter(df_subset_photo["BJD"],df_subset_photo["mag_scaled"])
plt.show()
'''

# write out as 'polished' photometry

In [ ]:
# cols BJD, mag, error

In [22]:
# remove rows of all NaNs

df_subset_photo_dropna = df_subset_photo.dropna(axis=0, how='all')

In [23]:
# write out only the relevant cols

df_subset_photo_dropna.to_csv(stem_public_sources + writeout_polished_file, 
                              columns=["BJD","Magnitude","Uncertainty"], index=False)
print(stem_public_sources + writeout_polished_file)

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/polished/av_peg_aavso_polished.txt
